<a href="https://colab.research.google.com/github/j3nguyen/user_item_embeddings/blob/master/NextAction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# NLTK for stopwords and stemming
from nltk.stem.porter import * 
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

# Plotting tools
# import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
import matplotlib

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [1]:
import keras

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Embedding, Activation
from keras.optimizers import Adam

In [0]:
df = pd.read_csv("events.csv")

In [15]:
df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [0]:
# reformat the data set
visitor_ids = df.visitorid.unique()

In [39]:
df.shape

(2756101, 5)

In [19]:
df.event.value_counts()

view           2664312
addtocart        69332
transaction      22457
Name: event, dtype: int64

In [0]:
action_to_idx = {"view": 0, "addtocart": 1, "transaction": 2}

In [68]:
# build the data set

a = df[df.visitorid == visitor_ids[342]].sort_values("timestamp").event.values
a = list(map(lambda x: action_to_idx[x],a))
a

[0, 0, 0]

In [0]:
visitor_activity = {}

for visitor in visitor_ids[:10000]:
  # get user's actions in order of time
  actions = df[df.visitorid == visitor].sort_values("timestamp").event.values
  
  if len(actions) >= 3:
    actions = list(map(lambda x: action_to_idx[x],actions)) [:3] 
    visitor_activity[visitor] = actions

In [0]:
#create the LSTM network
hidden_size = 64
num_actions = len(action_to_idx)
input_length = 2

model = Sequential()
model.add(Embedding(
    input_dim=num_actions,
    output_dim=hidden_size, #hidden layer dimension
    input_length=input_length,
    trainable=True
  )
)

model.add(LSTM(hidden_size, return_sequences=True))
model.add(Dropout(rate=0.5))
model.add(Activation('softmax'))


In [0]:
opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.01)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [105]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 2, 64)             192       
_________________________________________________________________
lstm_7 (LSTM)                (None, 2, 64)             33024     
_________________________________________________________________
dropout_7 (Dropout)          (None, 2, 64)             0         
_________________________________________________________________
activation_7 (Activation)    (None, 2, 64)             0         
Total params: 33,216
Trainable params: 33,216
Non-trainable params: 0
_________________________________________________________________


In [72]:
max(list(map(lambda x: len(x),visitor_activity.values())))

2252

In [0]:
from keras.utils import to_categorical

In [0]:
actions = visitor_activity.values()
X_train = to_categorical(np.array(list(map(lambda x: x[:-1], actions)))) # one-hot-encode
y_train = to_categorical(np.array(list(map(lambda x: x[1:], actions)))) # shift sequence on step forward


In [123]:
X_train.shape

(2871, 2, 3)

In [124]:
h = model.fit(X_train, y_train, epochs=5, batch_size=128, validation_split=0.1, verbose=1)

ValueError: ignored